In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
%store -r model
%store -r X
%store -r y
%store -r X_test
%store -r y_test
%store -r CNN_LSTM_model

In [3]:
from tensorflow.keras.models import Model

pretrained_model = model
# Xóa lớp cuối cùng của mô hình đã được huấn luyện trước đó
pretrained_model.pop()

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp phân loại mới
num_new_classes = 5  # Số lớp ứng dụng mới cần phân loại
pretrained_model.add(keras.layers.Dense(num_new_classes, activation='softmax', name='new_dense_layer'))

# Đóng băng trọng số của các lớp trong mô hình đã được huấn luyện trước đó
for layer in pretrained_model.layers[:-1]:
    layer.trainable = False

# In thông tin mô hình mới
pretrained_model.summary()

# Biên dịch mô hình mới
pretrained_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 20, 128, 32)       832       
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 128, 32)       25632     
                                                                 
 max_pooling2d (MaxPooling2  (None, 10, 64, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 10, 64, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 64, 64)        51264     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 64, 64)        102464    
                                                        

In [ ]:
pretrained_model.fit(X, y, epochs=10, batch_size=16)

Epoch 1/10
823/823 [==============================] - 32s 38ms/step - loss: 1.6464 - accuracy: 0.6480
Epoch 2/10
823/823 [==============================] - 29s 35ms/step - loss: 0.5309 - accuracy: 0.7883
Epoch 3/10
823/823 [==============================] - 30s 36ms/step - loss: 0.4856 - accuracy: 0.8157
Epoch 4/10
823/823 [==============================] - 30s 37ms/step - loss: 0.4763 - accuracy: 0.8175
Epoch 5/10
785/823 [===========================>..] - ETA: 1s - loss: 0.4795 - accuracy: 0.8096

In [ ]:
import numpy as np

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = pretrained_model.evaluate(X_test, y_test)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = pretrained_model.predict(X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)